## Import modules

In [1]:
import pdb
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import nhanes as nhanes

%matplotlib notebook

## Settings

In [2]:
DATA_PATH = '/c/users/josh_/Programming/Opportunistic/data/CDC/NHANES/'
DATASET = 'cancer'

### Note: 
The code below loads each dataset: dataset_features, dataset_targets

Here, all datasets are defined explicitly (see nhanes.py).

In [3]:
ds = nhanes.Dataset(DATA_PATH)
ds.load_cancer()
n_fe = ds.features.shape[1]
n_classes = 2

indx = np.argwhere(ds.targets != 3)
dataset_features = ds.features[indx.flatten()]
dataset_targets = ds.targets[indx.flatten()]

Processing: AUXTYM_D.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Processing: EPH_F.XPT

Processing: CBC_E.XPT

Processing: WPIN_D.XPT

Processing: SSTOCA_H.XPT

...

Processing: OCQ_F.XPT

Processing: SSQ.XPT

Processing: SMQ_E.XPT

Processing: SXQ_C.XPT                                                                                

Processing: WHQ_H.XPT

## Train/Test Separation

In [6]:
perm = np.random.permutation(dataset_targets.shape[0])
dataset_features = dataset_features[perm]
dataset_targets = dataset_targets[perm]

def get_batch(n_size, phase):
    # select indices
    n_samples = dataset_features.shape[0]
    n_classes = int(dataset_targets.max() + 1)
    if phase == 'test':
        inds_sel = np.arange(0, int(n_samples*0.15), 1)
    elif phase == 'validation':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.15), int(n_samples*0.30), 1)
    elif phase == 'train':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.30), n_samples, 1)
    else:
        raise NotImplementedError
    inds_sel = np.random.permutation(inds_sel)
    batch_inds = []
    for cl in range(n_classes):
        inds_cl = inds_sel[dataset_targets[inds_sel] == cl]
        batch_inds.extend(inds_cl[:n_size//n_classes])
    batch_inds = np.random.permutation(batch_inds)
    
    return dataset_features[batch_inds], dataset_targets[batch_inds]
    
features_trn, targets_trn = get_batch(n_size=5000, phase='train')
features_tst, targets_tst = get_batch(n_size=1000, phase='test')

## Classification

In [7]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_RFC', accu)

print(classification_report(targets_tst, preds_tst))

clf = SVC(gamma='auto')
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_SVC', accu)

print(classification_report(targets_tst, preds_tst))

clf = LogisticRegression(solver='lbfgs', max_iter=200)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_LR', accu)

print(classification_report(targets_tst, preds_tst))


accu_tst_RFC 0.727
              precision    recall  f1-score   support

           0       0.71      0.77      0.74       500
           1       0.75      0.68      0.71       500

   micro avg       0.73      0.73      0.73      1000
   macro avg       0.73      0.73      0.73      1000
weighted avg       0.73      0.73      0.73      1000

accu_tst_SVC 0.726
              precision    recall  f1-score   support

           0       0.71      0.76      0.74       500
           1       0.74      0.69      0.72       500

   micro avg       0.73      0.73      0.73      1000
   macro avg       0.73      0.73      0.73      1000
weighted avg       0.73      0.73      0.73      1000

accu_tst_LR 0.732
              precision    recall  f1-score   support

           0       0.72      0.76      0.74       500
           1       0.74      0.71      0.73       500

   micro avg       0.73      0.73      0.73      1000
   macro avg       0.73      0.73      0.73      1000
weighted avg      

In [16]:
print(classification_report(targets_tst, preds_tst))

              precision    recall  f1-score   support

           0       0.73      0.79      0.76       500
           1       0.77      0.71      0.74       500

   micro avg       0.75      0.75      0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.75      0.75      0.75      1000

